##### Copyright 2024 Google LLC.

In [ ]:
# @title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fine-tuning Gemma using Unsloth

Welcome to this step-by-step guide on fine-tuning the [Gemma](https://huggingface.co/google/gemma-2b) using [Unsloth](https://unsloth.ai/).


[**Gemma**](https://ai.google.dev/gemma) is a family of lightweight, state-of-the-art open models from Google, built from the same research and technology used to create the Gemini models. They are text-to-text, decoder-only large language models, available in English, with open weights, pre-trained variants, and instruction-tuned variants. Gemma models are well-suited for a variety of text generation tasks, including question answering, summarization, and reasoning. Their relatively small size makes it possible to deploy them in environments with limited resources such as a laptop, desktop or your own cloud infrastructure, democratizing access to state of the art AI models and helping foster innovation for everyone.

[**Unsloth**](https://unsloth.ai/) is a Python framework developed for finetuning large language models like Gemma 2.4x faster, using 58% less VRAM, and with no degradation in accuracy.


In this notebook, you will learn how to finetune Gemma 2 models using **Unsloth** in a Google Colab environment. You'll install the necessary packages, finetune the model, and run a sample inference.

<table align="left">
<td>
 <a target="_blank" href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/Gemma/[Gemma_2]Finetune_with_Unsloth.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
</td>
</table>

## Setup

### Select the Colab runtime
To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the Gemma model. In this case, you can use a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

### Hugging Face setup

**Before you dive into the tutorial, let's get you set up with Hugging Face. This is needed to upload the finetuned model into Hugging Face Hub.**

1. **Hugging Face Account:**  If you don't already have one, you can create a free Hugging Face account by clicking [here](https://huggingface.co/join).
2. **Hugging Face Token:**  Generate a Hugging Face access (with `write` permission) token by clicking [here](https://huggingface.co/settings/tokens). You'll need this token later in the tutorial.

**Once you've completed these steps, you're ready to move on to the next section where you'll set up environment variables in your Colab environment.**


### Configure your HF token

Add your Hugging Face token to the Colab Secrets manager to securely store it.

1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel. <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>
2. Create a new secret with the name `HF_TOKEN`.
3. Copy/paste your token key into the Value input box of `HF_TOKEN`.
4. Toggle the button on the left to allow notebook access to the secret.

In [6]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

### Install dependencies

You'll need to install a few Python packages and dependencies for Unsloth.

Run the following cell to install or upgrade the necessary packages:

In [7]:
#   Install Unsloth library
! pip install -q unsloth --upgrade --no-cache-dir

# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps -q packaging ninja einops "flash-attn>=2.6.3"

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

## Finetuning Gemma 2 using Unsloth library

### Initializing Gemma 2 model

Unsloth library supports a variety of open-source LLMs including Gemma. For this notebook, you will use Gemma 2's 2b model. You can load the Gemma 2 model in Unsloth using the `FastLanguageModel` class. To know more about the other variants of Gemma 2 model provided by Unsloth, visit the [Unsloth's supported models documentation](https://docs.unsloth.ai/get-started/all-our-models).

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b",
    max_seq_length = max_seq_length,
    dtype = None,        # None for auto detection.
    load_in_4bit = True, # Use 4bit quantization to reduce memory usage.
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.11: Fast Gemma2 patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

### Load a dataset

For this guide, you'll use an existing dataset from Hugging Face. You can replace it with your own dataset if you prefer.

The dataset chosen for this guide is [**yahma/alpaca-cleaned**](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a clean version of the original Alpaca dataset by Stanford. The Alpaca dataset is a collection of over 50,000 instructions and demonstrations that can be used to fine-tune language models to better understand and follow instructions.

**Credits:** **https://huggingface.co/yahma**

In [ ]:
from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Let's look at a few samples to understand the data.

In [ ]:
dataset[15]

{'output': "The motherboard, also known as the mainboard or system board, is the central printed circuit board in a computer. It serves as the backbone or foundation for a computer, connecting all the different components such as the CPU, RAM, storage drives, expansion cards, and peripherals. The motherboard manages communication and data transfer between these components, allowing them to work together and perform their designated tasks.\n\nThe motherboard also includes important circuitry such as the power regulation circuit that provides power to the different components, and the clock generator which synchronizes the operation of these components. It also contains the BIOS (basic input/output system), which is a firmware that controls the boot process and provides an interface for configuring and managing the computer's hardware. Other features on a motherboard may include built-in networking, audio, and video capabilities.\n\nOverall, the function of a computer motherboard is to p

In [ ]:
from google.colab import data_table
import pandas as pd

# Enable interactive DataFrame display
data_table.enable_dataframe_formatter()

# Convert the 'train' split to a Pandas DataFrame
df_train = pd.DataFrame(dataset)

# Select the first 5 rows
df_train.head(5)

,output,input,instruction
0,1. Eat a balanced and nutritious diet: Make su...,,Give three tips for staying healthy.
1,"The three primary colors are red, blue, and ye...",,What are the three primary colors?
2,An atom is the basic building block of all mat...,,Describe the structure of an atom.
3,There are several ways to reduce air pollution...,,How can we reduce air pollution?
4,I had to make a difficult decision when I was ...,,Pretend you are a project manager of a constru...


### Set the prompt template

Here you will define the Alpaca prompt template. This template has 3 sections:
- Instruction
- Input
- Response

In [ ]:
alpaca_prompt_template = """Below is an instruction that describes a task, paired with an
input that provides further context. Write a response that appropriately
completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # EOS_TOKEN is necessary.

### Define the formatting function

The formatting function applies the template created above to each row in the dataset and converts it into a format suited for training.

In [ ]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    # EOS_TOKEN is necessary, otherwise your generation will go on forever!
    texts = [alpaca_prompt_template.format(instruction, input, output) + EOS_TOKEN
                                  for instruction, input, output in
                                  zip(instructions, inputs, outputs)]
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

### Set LoRA configuration

LoRA (Low-Rank Adaptation) allows for efficient fine-tuning by adapting only a subset of model parameters.

Here, you set the following parameters:
- `r` to 16, which controls the rank of the adaptation matrices.
- `lora_alpha` to 16 for scaling.
- `lora_dropout` to 0 since it is optimized.

To know more about LoRA parameters and their effects, check out the [LoRA parameters encyclopedia](https://github.com/unslothai/unsloth/wiki#lora-parameters-encyclopedia).

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # LoRA attention dimension
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,  # Alpha parameter for LoRA scaling
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # Rank stabilized LoRA
    loftq_config = None, # LoRA-Fine-Tuning-Aware Quantization
)

Unsloth 2024.11.11 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


### Set training configuration

Set up the training arguments that define how the model will be trained.

Here, you'll define the following parameters:

- For training and evaluation:
  - `output directory`
  - `max steps`
  - `batch sizes`

- To optimize the training process:
  - `learning rate`
  - `optimizer`
  - `learning rate scheduler`

**Note:** `max_steps` is set as 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run.

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 5,
    # num_train_epochs = 1, # Set this for 1 full training run.
    max_steps = 60,
    learning_rate = 2e-4,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 1,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 42,
    output_dir = "outputs",
    report_to = "none",
)

<a name="Train"></a>
### Train the model

[Huggingface's TRL](https://huggingface.co/docs/trl/index) offers a user-friendly API for building SFT models and training them on your dataset with just a few lines of code. Here you will use Huggingface TRL's `SFTTrainer` class to train the model. This class inherits from the `Trainer` class available in the Transformers library, but is specifically optimized for supervised fine-tuning (instruction tuning). Read more about SFFTrainer from the [official TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer).

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    # Setting packing as False can speed up training five times
    # for short sequences.
    packing = False,
    args = training_args
)

Map (num_proc=2):   0%|          | 0/51760 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Now, let's start the fine-tuning process by calling `trainer.train()`, which uses `SFTTrainer` to handle the training loop, including data loading, forward and backward passes, and optimizer steps, all configured according to the settings you've provided.

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 20,766,720


Step,Training Loss
1,1.874800
2,2.086800
3,2.080000
4,2.015800
5,1.535900
6,1.430500
7,1.328700
8,1.366300
9,1.179300
10,1.110300


### Save the model locally

After training is complete, save the fine-tuned model by calling `save_pretrained(new_model)`. This saves the model weights and configuration files to the directory specified by `new_model` (**gemma_ft_unsloth**). You can reload and use the fine-tuned model later for inference or further training.

In [ ]:
new_model = "gemma_ft_unsloth"
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

('gemma_ft_unsloth/tokenizer_config.json',
 'gemma_ft_unsloth/special_tokens_map.json',
 'gemma_ft_unsloth/tokenizer.model',
 'gemma_ft_unsloth/added_tokens.json',
 'gemma_ft_unsloth/tokenizer.json')

### Push to Hugging Face Hub

You can use the model's `push_to_hub` method to upload the model to Hugging Face Hub.

**Note**: In the following code snippets, replace "your_hf_username" to your Hugging Face username.

In [ ]:
# Push the trained model to Hub
model.push_to_hub("harjeetkr/gemma_ft_unsloth")

If you only want to save the Lora adapters, uncomment and run the code below.

In [ ]:
#model.push_to_hub_merged("your_hf_username/gemma_ft_lora", tokenizer,
#                         save_method = "lora", token = os.environ["HF_TOKEN"])

If you want to save the model as GGUF, uncomment and run the code below. To check the different quantization methods supported by Unsloth, visit
[Unsloth wiki's Save as GGUF section](https://github.com/unslothai/unsloth/wiki#saving-to-gguf).

In [ ]:
#model.push_to_hub_gguf("your_hf_username/gemma_ft_q4_k_m",
#                       tokenizer, quantization_method = "q4_k_m")

To know more about the different formats available in Unsloth, check out [Unsloth wiki's saving models section](https://github.com/unslothai/unsloth/wiki#saving-models-to-16bit-for-vllm).

## Inference

### Prompt using the newly fine-tuned model


Now that you've finally fine-tuned your custom Gemma model, let's reload the LoRA adapter weights and tokenizer to finally prompt using it and also verify if it's really working as intended.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = new_model, # Your finetuned model name
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.11.11: Fast Gemma2 patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma2ForCausalLM(
      (model): Gemma2Model(
        (embed_tokens): Embedding(256000, 2304, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma2DecoderLayer(
            (self_attn): Gemma2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2304, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2304, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora

Now, test the fine-tuned model with a sample prompt by first using the tokenizer to generate the input ids, and then relying on the reloaded fine-tuned model to generate a response using `model.generate()`.

Instead of waiting the entire time, you can view the generation token by token by using a `TextStreamer` for continuous inference.

In [ ]:
from transformers import TextStreamer

inputs = tokenizer(
[
    alpaca_prompt_template.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<bos>Below is an instruction that describes a task, paired with an
input that provides further context. Write a response that appropriately
completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
The next number in the Fibonacci sequence is 13.<eos>


Congratulations! You've successfully fine-tuned Gemma using Unsloth. You've covered the entire process, from setting up the environment to training and testing the model.

## What's next?
Your next steps could include the following:

- **Experiment with Different Datasets**: Try fine-tuning on other datasets in [Hugging Face](https://huggingface.co/docs/datasets/en/index) or your own data to adapt the model to various tasks or domains.

- **Tune Hyperparameters**: Adjust training parameters (e.g., learning rate, batch size, epochs, LoRA settings) to optimize performance and improve training efficiency.

- **Save model in different formats**: Check the different saving formats provided by Unsloth and use it in `llama.cpp` or a UI based system like `GPT4All`.

By exploring these activities, you'll deepen your understanding and further enhance your fine-tuned Gemma model. Happy experimenting!